In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

### Load the dataset

In [ ]:
(train_ds, validation_ds, test_ds), info = tfds.load(
    "caltech101",
    split=["train", "test[:50%]", "test[50%:]"],
    as_supervised=True,
    with_info=True
)

In [ ]:
# Check sizes of datasets
print(train_ds.cardinality())
print(validation_ds.cardinality())
print(test_ds.cardinality())


In [ ]:
print(f"Description: {info.description}")

In [ ]:
# check datatype
for image, label in train_ds.take(1):
  print(f"{image.dtype}, {label.dtype}")

### Some Data Exploration

In [ ]:
# Show some examples
plt.figure(figsize=(10,10))

for i, (image, label) in enumerate(train_ds.take(9)):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image)
  plt.title(f"{int(label)}, shape: {image.shape}")
  plt.axis("off")

In [ ]:
# Resize the images
IMAGE_SIZE=(224,224)
def resize(image, height=224, width=224):
  return tf.keras.layers.Resizing(height, width)(image)

train_ds = train_ds.map(lambda image, label : (resize(image), label))
validation_ds = validation_ds.map(lambda image, label : (resize(image), label))
test_ds = test_ds.map(lambda image, label : (resize(image), label))

In [ ]:
# Toon sommige afbeeldingen opnieuw. Merk op. Resizing layer retourneert floats en dat
# vindt imshow by default niet leuk
plt.figure(figsize=(10,10))

for i, (image, label) in enumerate(train_ds.take(9)):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image.numpy().astype(np.int32))
  plt.title(f"{int(label)}, shape: {image.shape}")
  plt.axis("off")

### Preprocess the data

In [ ]:
# We willen mobile net V2 gebruiken.
# Batch de dataset (batch_size = 32)
# Pas de gepaste preprocessing functie toe op alle elementen van de dataset.
# Shuffle de training data
preprocess = tf.keras.applications.mobilenet_v2.preprocess_input

BATCH_SIZE=32
train_ds = (train_ds
    .map(lambda x, y : (preprocess(x), y))
    .shuffle(1000, seed=42)
    .batch(BATCH_SIZE)
    .prefetch(1)
)
validation_ds = (validation_ds
    .map(lambda x, y : (preprocess(x), y))
    .batch(BATCH_SIZE)
)
test_ds = (test_ds
    .map(lambda x, y : (preprocess(x), y))
    .batch(BATCH_SIZE)
)

### Apply data augmentation

In [ ]:
# Apply data data augmentation
# Random horizontal flip
# Random small rotation
# Random zoom
data_augmentation = tf.keras.Sequential(
    [tf.keras.layers.RandomFlip("horizontal"),
     tf.keras.layers.RandomRotation(20/360, fill_mode='constant'),
     tf.keras.layers.RandomZoom(height_factor=(-0.15,0.15), fill_mode="constant"),
    ]
)

In [ ]:
# Bekijk voorbeelden van afbeeldingen die augmented zijn.
for images, labels in validation_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[2]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow( (augmented_image[0].numpy() + 1)/2)
        plt.title(int(labels[0]))
        plt.axis("off")

### Load the base model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(*IMAGE_SIZE, 3),
    include_top=False,
    weights="imagenet",
)

In [ ]:
# Vind de vorm van de laatste laag door eenvoudigweg een batch door het model
# te jagen en te kijken naar de shape
for batch, label in train_ds.take(1):
  print(base_model(batch).shape)

In [ ]:
base_model.trainable=False

In [ ]:
input_ = tf.keras.layers.Input(shape=(*IMAGE_SIZE, 3))
x = data_augmentation(input_)
x = input_
x = base_model(x, training=False) # Add training = False
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(rate=0.4)(x)
output = tf.keras.layers.Dense(units=102, activation='softmax')(x)
model = tf.keras.Model(inputs=[input_], outputs=[output])

In [ ]:
model.summary()

### Train the model

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3)

In [ ]:
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    min_delta=0.001,
    restore_best_weights=True
)

In [ ]:
history = model.fit(train_ds, validation_data=validation_ds,
                    epochs=100,
                    callbacks=[early_stopping])

In [ ]:
def plot_learning_curves(history):
    plt.figure(figsize=(8, 5))
    for key, style in zip(history.history, ["r-o", "r-*", "b-o", "b-*"]):
        epochs = np.array(history.epoch)
        plt.plot(epochs + 1, history.history[key], style, label=key)
    plt.xlabel("Epoch")
    plt.axis([1, len(history.history['loss']), 0., 1])
    plt.legend(loc="lower left")
    plt.grid()

In [ ]:
plot_learning_curves(history)

In [ ]:
base_model.summary()

In [ ]:
# Find index of "block_13_expand"
[layer.name for layer in base_model.layers].index("block_13_expand")

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:116]:
  layer.trainable = False

In [ ]:
fine_tuning_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)


In [ ]:
model.compile(
    optimizer=fine_tuning_optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_ds, validation_data=validation_ds,
                    epochs=100,
                    callbacks=[early_stopping])

### Evaluate the model

In [ ]:
model.evaluate(test_ds)

### Top K accuracy

In [ ]:
top5 = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5")
top3 = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="top3")

In [ ]:
model.compile(
    optimizer=fine_tuning_optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", top3, top5]
)

In [ ]:
model.evaluate(test_ds)